#### Functions

In [1]:
def glue_tokens(tokens, order):
    """A useful way of glueing tokens together for
    Kneser Ney smoothing and other smoothing methods
    
    :param: order is the order of the language model
        (1 = unigram, 2 = bigram, 3 =trigram etc.)
    """
    return '{0}@{1}'.format(order,' '.join(tokens))

def unglue_tokens(tokenstring, order):
    """Ungluing tokens glued by the glue_tokens method"""
    if order == 1:
        return [tokenstring.split("@")[1].replace(" ","")]
    return tokenstring.split("@")[1].split(" ")

def tokenize_sentence(sentence, order):
    """Returns a list of tokens with the correct numbers of initial
    and end tags (this is meant ot be used with a non-backoff model!!!)
    
    :sentence: a string of text
    :param: order is the order of the language model
        (1 = unigram, 2 = bigram, 3 =trigram etc.)
    """
    tokens = sentence.split()
    tokens = ['<s>'] * (order-1) + tokens + ['</s>']
    return tokens

def find_unigrams(filename, MDF): # ex 1 ; #finds the unigram with a set minium document frequency. Key values used as a vocab
    dataSource = open(filename, 'r')
    unigrams = Counter()

    for line in dataSource:
        words = tokenize_sentence(line,1)
        for w in words:
            unigrams[w] += 1

    #eremoves words under the MDF.
    unigramToRemove = [k for k in unigrams if unigrams[k] < MDF] #Combind training data pass 1 and 2 into one pass.
    for w in unigramToRemove:
        unigrams["<unk/>"] += unigrams[w] #allow for variable MDF
        del unigrams[w]

    return unigrams

def readInAndUnkOOV(filename, vocabCount): # exercise 1
    dataSource = open(filename, 'r')
    dataSansOOV = [];
    for line in dataSource: #reads in a files and UNKifies words that are not in the vocab.
        words = tokenize_sentence(line,1)
        words[:] = ['<unk/>' if not(x in vocabCount) else x for x in words]
        dataSansOOV.append(words)
    
    return dataSansOOV
        
def readInUnkOOVRejoin(filename, vocabCount): #a readIn Varient. UNKifies data and joins it back into an list of lists.
    dataSansOOV = readInAndUnkOOV(filename, vocabCount)
    joinedData = [];
    for x in dataSansOOV:
        joinedData.append(' '.join(x))
    return joinedData


#calculates the bigrams in a file with MDF defined by the vocab.    
def find_bigrams(filename, vocab, MDF): # exercise 2. Vocab represented by Unigram
    dataSource = open(filename, 'r')
    dataSansOOV = [];
    for line in dataSource:
        words = line.split(" ")
        words[:] = ['<unk/>' if not(x in vocab) else x for x in words]
        dataSansOOV.append(' '.join(words)) #rejoins the words now Unkified        
    

    bigrams = Counter() # a counter for how many times a given bigram sequence w_i-1,w_i occurs
    bigram_context = Counter() # a counter for how many times each word is used as a context word w_i-1 (so will include the start symbol)
    delta = 1  # delta is order - 1
    for line in dataSansOOV:
        words = tokenize_sentence(line, 2)  # tokenize sentence with the order 2 as the parameter
        for i in range(delta, len(words)):
            context = words[i-delta:i]
            target = words[i]
            ngram = context + [target]
            bigrams[glue_tokens(ngram, 2)] +=1
            bigram_context[glue_tokens(context, 1)] += 1
    print(len(bigrams.keys()), "different bigrams")
    print(len(bigram_context.keys()), "different bigram contexts (and unigrams) observed")

    return bigrams, bigram_context
        
def prob_bigram_add_one(ngram, i): # computes add one probability of an input ngram
    """A simple function to compute the 
    MLE probability estimation based on the counts.
    Follows the equation:
    C(w_i-1, w_i)/C(w_i-1)
    
    Dictionaries bigrams and bigram_context are global variables.

    """
    numerator = bigrams[glue_tokens(ngram, 2)]
    denominator = bigram_context[glue_tokens(ngram[:1], 1)]
    prob = (numerator + 1) / (denominator + i)
    return prob

def prob_bigram_add_k(ngram, i, k): #computes the add k probability of an input ngram
    """A simple function to compute the 
    MLE probability estimation based on the counts.
    Follows the equation:
    C(w_i-1, w_i)/C(w_i-1)
    
    Dictionaries bigrams and bigram_context are global variables.

    """
    numerator = bigrams[glue_tokens(ngram, 2)]
    denominator = bigram_context[glue_tokens(ngram[:1], 1)]
    prob = (numerator + k) / (denominator + k*i)
    return prob

def prob_bigram_MLE(ngram): #example method
    """A simple function to compute the 
    MLE probability estimation based on the counts.
    Follows the equation:
    C(w_i-1, w_i)/C(w_i-1)
    
    Dictionaries bigrams and bigram_context are global variables.

    """
    numerator = bigrams[glue_tokens(ngram, 2)]
    denominator = bigram_context[glue_tokens(ngram[:1], 1)]
    prob = numerator / denominator
    return prob

def readIn(filename, mode): # execise 2 mostly Reads in a file and splits it.
    dataSource = open(filename, 'r')
    data = [];
    if mode == 1:
        for line in dataSource:
            words = tokenize_sentence(line,1)
            data.append(words)
    else:
        count = 0
        for line in dataSource:
            words = tokenize_sentence(line,1)
            data.append(words)
            count += 1
            if count > 1000:
                break
    return data

def kneser_ney_ngram_prob(ngram, discount, order): 
    """KN smoothed ngram probability from Goodman 2001.
    This is run at test time to calculate the probability
    of a given n-gram or a given order with a given discount.
    
    ngram :: list of strings, the ngram
    discount :: float, the discount used (lambda)
    order :: int, order of the model
    """
    # First, calculate the unigram prob of the last token 
    # If we've never seen it at all, it will 
    # have no probability as a numerator
    uni_num = ngram_numerator_map.get(glue_tokens(ngram[-1], 1))
    if not uni_num: # if no value found in dict, make it 0
        uni_num = 0
    probability = previous_prob = float(uni_num) / float(unigram_denominator)
    
    # Given <unk/> should have been used in place of unknown words before passing
    # to this method,
    # probability should be non-zero
    if probability == 0.0:
        print("0 prob for unigram!")
        print(glue_tokens(ngram[-1], 1))
        print(ngram)
        print(ngram_numerator_map.get(glue_tokens(ngram[-1], 1)))
        print(unigram_denominator)
        raise Exception

    # Compute the higher order probs (from 2/bi-gram upwards) and interpolate them
    for d in range(2,order+1):
        # Get the number of times this denominator has been seen as one
        # For bigrams this is the number of different continuation types counted
        ngram_den = ngram_denominator_map.get(glue_tokens(ngram[-(d):-1], d))
        if not ngram_den: # if no value found in dict, make it 0
            ngram_den = 0
        if ngram_den != 0: 
            ngram_num = ngram_numerator_map.get(glue_tokens(ngram[-(d):], d))
            if not ngram_num: # if no value found in dict, make it 0
                ngram_num = 0
            if ngram_num != 0:
                current_prob = (ngram_num - discount) / float(ngram_den)
            else:
                current_prob = 0.0
            nonzero = ngram_non_zero_map.get(glue_tokens(ngram[-(d):-1], d))
            if not nonzero: # if no value found in dict, make it 0
                nonzero = 0
            # interpolate with previous probability of lower orders calculated
            # so far
            current_prob += nonzero * discount / ngram_den * previous_prob
            previous_prob = current_prob
            probability = current_prob
        else:
            #if this context (e.g. bigram contect for trigrams) has never been seen, 
            #then we can only get the last order with a probability (e.g. unigram)
            #and halt
            probability = previous_prob
            break
    return probability

def ngrams_interpolated_kneser_ney(tokens,
                                   order,
                                   ngram_numerator_map,
                                   ngram_denominator_map,
                                   ngram_non_zero_map,
                                   unigram_denominator):
    """Function used in n-gram language model training
    to count the n-grams in tokens and also record the
    lower order non -ero counts necessary for interpolated Kneser-Ney
    smoothing.
    
    Taken from Goodman 2001 and generalized to arbitrary orders"""
    for i in range(order-1,len(tokens)): # tokens should have a prefix of order - 1
        #print i
        for d in range(order,0,-1): #go through all the different 'n's
            if d == 1:
                unigram_denominator += 1
                ngram_numerator_map[glue_tokens(tokens[i],d)] += 1
            else:
                den_key = glue_tokens(tokens[i-(d-1) : i], d)
                num_key = glue_tokens(tokens[i-(d-1) : i+1], d)
    
                ngram_denominator_map[den_key] += 1
                # we store this value to check if it's 0
                tmp = ngram_numerator_map[num_key]
                ngram_numerator_map[num_key] += 1 # we increment it
                if tmp == 0: # if this is the first time we see this ngram
                    #number of types it's been used as a context for
                    ngram_non_zero_map[den_key] += 1
                else:
                    break 
                    # if the ngram has already been seen
                    # we don't go down to lower order models
    return ngram_numerator_map, ngram_denominator_map, ngram_non_zero_map, unigram_denominator


#### Exercise 1

In [6]:
from collections import Counter
from math import log

fileTraining = 'switchboard_lm_train.txt'
fileTesting = 'switchboard_lm_test.txt'
fileHeldOut = 'switchboard_lm_heldout.txt'

unigrams = find_unigrams(fileTraining,2) #the key values of the unigram are also a vocab.

unigram_total = sum(unigrams.values());
print(len(unigrams), "Unique unigrams observed")
print("unigram total", unigram_total)

testData = readInAndUnkOOV(fileTesting, unigrams) # Reads int he testdata and UNKifies OOV words.

s = 0  # total neg log prob mass for cross entropy
N = 0 # total number of words for normalizing s 
for sent in testData:
    # get the unigram model based probability of each sentence
    sent_s = 0  # recording non-normalized entropy for this sentence
    sent_N = 0  # total number of words in this sentence (for normalization)
    for w in sent:
        #print(str(unigrams[w]) + " " + w)
        prob = unigrams[w]/unigram_total
        s += -log(prob, 2) # add the neg log prob to s
        sent_s += -log(prob, 2)  # add the neg log prob to sent_s
        N += 1 # increment the number of total words
        sent_N += 1 # increment the number of total words in this sentence
    sent_cross_entropy = sent_s/sent_N
    sent_perplexity = 2 ** sent_cross_entropy
    #print(sent, "cross entropy:", sent_cross_entropy, "perplexity:", sent_perplexity)
cross_entropy = s/N
perplexity = 2 ** cross_entropy
print("Test corpus cross entropy", cross_entropy)
print("Test corpus perplexity", perplexity)


12730 Unique unigrams observed
unigram total 1306313
Test corpus cross entropy 8.287702715147693
Test corpus perplexity 312.4979066155026


#### Exercise 2

In [7]:
from collections import Counter
from math import log

fileTraining = 'switchboard_lm_train.txt'
fileTesting = 'switchboard_lm_test.txt'
fileHeldOut = 'switchboard_lm_heldout.txt'

unigrams = find_unigrams(fileTraining,2) #used to contruct MDF 2 Vocab from training set.
bigrams, bigram_context = find_bigrams(fileTraining, unigrams,2) #trains a bigram

#print(unigrams["<unk/>"])
#add one smoothing implementation

testData = readIn(fileTesting, 1) #reads in test data

s = 0
N = 0
delta = 1

for sent in testData: # standard calculation of the cross entropy
    for i in range(delta, len(sent)):
        context = sent[i-delta:i]
        target = sent[i]
        ngram = context + [target]
        prob = prob_bigram_add_one(ngram, len(unigrams))
        s += -log(prob,2)
        N += 1
        
cross_entropy = s/N
perplexity = 2 ** cross_entropy
print(str(cross_entropy) + " Cross Entropy") #10.061359048452715 Cross Entropy
print(str(perplexity) + " Perplexity") #1068.4910057009279 Perplexity

183922 different bigrams
12225 different bigram contexts (and unigrams) observed
7898
10.061359048452715 Cross Entropy
1068.4910057009279 Perplexity


#### Exercise 3

In [ ]:
from collections import Counter
from math import log

fileTraining = 'switchboard_lm_train.txt'
fileTesting = 'switchboard_lm_test.txt'
fileHeldOut = 'switchboard_lm_heldout.txt'

unigrams = find_unigrams(fileTraining,2) #used to contruct MDF 2 Vocab from training set.
bigrams, bigram_context = find_bigrams(fileTraining, unigrams,2) #trains a bigram

#add one smoothing implementation

testData = readIn(fileTesting, 1)

s = 0
N = 0
delta = 1

for sent in testData: # standard calculation of the cross entropy
    for i in range(delta, len(sent)):
        context = sent[i-delta:i]
        target = sent[i]
        ngram = context + [target]
        prob = prob_bigram_add_k(ngram, len(unigrams), 100) #the prob of a bigram using add k.
        s += -log(prob,2)
        N += 1
        
cross_entropy = s/N
perplexity = 2 ** cross_entropy
print(str(cross_entropy) + " Cross Entropy")
print(str(perplexity) + " Perplexity")

'''
the perplexity drops the lower the k value
k = 0.2
9.404859340160199 Cross Entropy
677.8674282013725 Perplexity

k = 0.4
9.639059948911317 Cross Entropy
797.3448153618554 Perplexity

k = 0.6
9.810956888083794 Cross Entropy
898.2398314874424 Perplexity

k = 0.8
9.947594955533061 Cross Entropy
987.4712641728472 Perplexity

k = 2
10.451833167515845 Cross Entropy
1400.6037767581658 Perplexity

k = 100 interested to see if the trend continues. (It does!)
12.723824005557153 Cross Entropy
6764.764546838359 Perplexity
'''

#### Exercise 4

In [ ]:
fileTraining = 'switchboard_lm_train.txt'
fileTesting = 'switchboard_lm_test.txt'
fileHeldOut = 'switchboard_lm_heldout.txt'

# Kneser-Ney smoothing
order = 4
discount = 0.9

unigram_denominator = 0
ngram_numerator_map = Counter() 
ngram_denominator_map = Counter() 
ngram_non_zero_map = Counter()

unigrams = find_unigrams(fileTraining,2) #unigram keys = vocab
corpus =  readInUnkOOVRejoin(fileTraining, unigrams)#create unkified training data.

# train the model

for line in corpus:
    tokens = tokenize_sentence(line, order)
    ngram_numerator_map, ngram_denominator_map, ngram_non_zero_map, unigram_denominator =\
            ngrams_interpolated_kneser_ney(tokens,
                                           order,
                                           ngram_numerator_map,
                                           ngram_denominator_map,
                                           ngram_non_zero_map,
                                           unigram_denominator)
    
heldoutCorpus = readInUnkOOVRejoin(fileHeldOut, unigrams) # read in and UNK OOV words in heldout corpus

s = 0
N = 0
delta = order - 1 

for sent in heldoutCorpus:
    temp = sent.split()
    #print(temp)
    for i in range(delta, len(temp)):
        context = temp[i-delta:i]
        target = temp[i]
        ngram = context + [target]
        #print(ngram)
        prob = kneser_ney_ngram_prob(ngram,discount,order);
        s += -log(prob,2)
        N += 1

cross_entropy = s/N
perplexity = 2 ** cross_entropy
print(str(cross_entropy) + " Cross Entropy")
print(str(perplexity) + " Perplexity")

testCorpus = readInUnkOOVRejoin(fileTesting, unigrams)

s = 0
N = 0
delta = order - 1

#probability calculation over the test corpus using kneser_ney
for sent in testCorpus:
    temp = sent.split()
    #print(temp)
    for i in range(delta, len(temp)):
        context = temp[i-delta:i]
        target = temp[i]
        ngram = context + [target]
        #print(ngram)
        prob = kneser_ney_ngram_prob(ngram,discount,order); 
        s += -log(prob,2)
        N += 1

cross_entropy = s/N
perplexity = 2 ** cross_entropy
print(str(cross_entropy) + " Cross Entropy")
print(str(perplexity) + " Perplexity")

'''
Discount, Order
0.2 3
7.320822904064247 Cross Entropy
159.87747735135233 Perplexity

0.4 3
6.893463689938024 Cross Entropy
118.88836231370918 Perplexity

0.6 3
6.676621625215499 Cross Entropy
102.29711336245768 Perplexity

0.8 3
6.568029012633623 Cross Entropy
94.8797966432974 Perplexity

0.2 4
7.975827045678766 Cross Entropy
251.74634908436195 Perplexity

0.4 4
7.246896062520934 Cross Entropy
151.89136684868024 Perplexity

0.6 4
6.862635118751278 Cross Entropy
116.37481940672485 Perplexity

0.8 4
6.648434145960116 Cross Entropy
100.317823734305 Perplexity

0.9 4                            The lowest perplexity
6.603638530964016 Cross Entropy
97.25082144272686 Perplexity

1.0 4 
6.708794374138775 Cross Entropy
104.6040118802196 Perplexity
0.2 5
8.3208471723169 Cross Entropy
319.7603334966111 Perplexity

0.4 5
7.454968793527132 Cross Entropy
175.4564037552623 Perplexity

0.6 5
6.9923544869937855 Cross Entropy
127.32346260634942 Perplexity

0.8 5
6.725611226436778 Cross Entropy
105.8304684089911 Perplexity

The cross entryopy is 6.64575074582797 and the Perplexity is 100.13140688195276.
On the test data when the order is 4 and the discount is 0.9
'''

